In [1]:
import datetime
import json
from langchain_community.chat_models import ChatOllama

MODEL_NAME = "llama3.1:8b"
llm = ChatOllama(model=MODEL_NAME, temperature=0.1)

def generate_roadmap_tool(skill, current_level="Beginner"):
    print(f"Generating roadmap for: {skill}...")
    
    prompt = f"""
    You are an Expert Curriculum Designer. 
    Create a step-by-step learning roadmap for: {skill}
    Target Level: {current_level} -> Advanced.
    
    Strictly output ONLY a JSON array of objects. Do not output any other text.
    Format:
    [
        {{"topic": "Topic Name", "hours_needed": 5, "description": "Brief summary", "search_query": "Best free resource for Topic Name"}}
    ]
    """
    
    try:
        response = llm.invoke(prompt)
        clean_json = response.content.replace("```json", "").replace("```", "").strip()
        roadmap = json.loads(clean_json)
        return roadmap
    except Exception as e:
        print(f"Error parsing roadmap: {e}")
        return []

def create_schedule_tool(roadmap_data, hours_per_day=2, start_date=None):
    print(f"Mapping to calendar ({hours_per_day} hrs/day)...")
    
    if start_date is None:
        start_date = datetime.date.today()
    else:
        start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d").date()
        
    schedule = []
    current_date = start_date
    
    for module in roadmap_data:
        topic = module['topic']
        needed = module.get('hours_needed', 2)
        
        days_required = (needed // hours_per_day) + (1 if needed % hours_per_day > 0 else 0)
        end_date = current_date + datetime.timedelta(days=max(1, days_required - 1))
        
        schedule.append({
            "topic": topic,
            "start_date": str(current_date),
            "end_date": str(end_date),
            "daily_commitment": f"{hours_per_day} hours",
            "focus": module.get('description', '')
        })
        
        current_date = end_date + datetime.timedelta(days=1)
        
    return schedule

def run_test():
    print("TESTING SKILL GRAPH & SCHEDULER")
    
    # 1. Generate the Skill Graph
    user_skill = "Docker"
    roadmap = generate_roadmap_tool(user_skill)
    
    if not roadmap:
        print("Failed to generate roadmap.")
        return

    print(f"\nROADMAP GENERATED ({len(roadmap)} Modules):")
    for module in roadmap:
        print(f"- {module['topic']} ({module['hours_needed']} hrs)")

    # 2. Map to Schedule
    print("\nGENERATING PERSONALIZED SCHEDULE...")
    final_schedule = create_schedule_tool(roadmap, hours_per_day=2)
    
    print("\nFINAL USER PLAN:")
    print(json.dumps(final_schedule, indent=2))

if __name__ == "__main__":
    run_test()

C:\Users\hari7\AppData\Local\Temp\ipykernel_40388\2620573396.py:6: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=MODEL_NAME, temperature=0.1)


TESTING SKILL GRAPH & SCHEDULER
Generating roadmap for: Docker...

ROADMAP GENERATED (9 Modules):
- Introduction to Docker (2 hrs)
- Installing and Configuring Docker (3 hrs)
- Docker Images and Containers (4 hrs)
- Dockerfile and Build Process (5 hrs)
- Docker Networking and Volumes (4 hrs)
- Docker Compose and Orchestration (6 hrs)
- Docker Security and Best Practices (5 hrs)
- Advanced Docker Topics (Cgroups, Namespaces, etc.) (8 hrs)
- Real-World Applications and Case Studies (10 hrs)

GENERATING PERSONALIZED SCHEDULE...
Mapping to calendar (2 hrs/day)...

FINAL USER PLAN:
[
  {
    "topic": "Introduction to Docker",
    "start_date": "2025-12-30",
    "end_date": "2025-12-31",
    "daily_commitment": "2 hours",
    "focus": "Learn the basics of Docker and its ecosystem."
  },
  {
    "topic": "Installing and Configuring Docker",
    "start_date": "2026-01-01",
    "end_date": "2026-01-02",
    "daily_commitment": "2 hours",
    "focus": "Install Docker on your local machine and le